In [ ]:
import os
import sys
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm, trange
import scipy.sparse as sp
# import wandb
# import weave

# Metrics

In [ ]:
# base_path = '../results/metrics/'
# nsms = os.listdir(base_path)
# methods = os.listdir('')
# nsm = 'random'
# method = 'sl2mf'
# path = f'../results/metrics/{nsm}/{method}'
# res_name = os.listdir(path)
# # res_name
# pd.read_csv(f'{path}/{res_name[1]}').loc[2,:].values[3:].astype(float)

In [ ]:
base_path = '../results/metrics'
# base_path = '../results/metrics_wo_compt'
summary_res_df = pd.DataFrame(columns=['AUROC', 'AUPR', 'F1', 'N10', 'N20', 'N50', 'R10', 'R20', 'R50', 'P10', 'P20', 'P50', 'M10', 'M20', 'M50'],
                              index = ['CV1,1','CV2,1','CV3,1','CV1,5','CV2,5','CV3,5','CV1,20','CV2,20','CV3,20','CV1,50','CV2,50','CV3,50'])
nsms = os.listdir(base_path)
res_dict = {}
for nsm in nsms:
    res_dict[nsm]={}
    methods = os.listdir(f'{base_path}/{nsm}')
    for method in methods:
        res_dict[nsm][method] = copy.deepcopy(summary_res_df)
        res_name = os.listdir(f'{base_path}/{nsm}/{method}')
        for n in res_name:
            res = pd.read_csv(f'{base_path}/{nsm}/{method}/{n}')
            res = res.loc[2,:].values[3:].astype(float)
            n_split = n.split('_')
            cvx,pnr = n_split[4],int(1/float(n_split[3]))
            x_id = str(cvx)+','+str(pnr)
            res_dict[nsm][method].loc[x_id,:] = res
        # res_dict[nsm][method] = res_dict[nsm][method][['AUROC', 'AUPR', 'F1', 'N10', 'N20', 'N50', 'R10', 'R20', 'R50', 'P10', 'P20', 'P50', 'M10', 'M20', 'M50']]
        res_dict[nsm][method].to_csv(f'../results/summary/summary_{nsm}_{method}.csv')


In [ ]:
all_col_name = ['Model', 'NSM']
for i in ['CV1,1','CV2,1','CV3,1','CV1,5','CV2,5','CV3,5','CV1,20','CV2,20','CV3,20','CV1,50','CV2,50','CV3,50']:
    for j in ['AUROC', 'AUPR', 'F1', 'N10', 'N20', 'N50', 'R10', 'R20', 'R50', 'P10', 'P20', 'P50', 'M10', 'M20', 'M50']:
        all_col_name.append(i+','+j)
len(all_col_name)

In [ ]:
# new_df = pd.DataFrame(columns=all_col_name)
score_values_df = pd.DataFrame(columns=all_col_name)
score_values = np.zeros((36,182), dtype=object)
score_values[:,1] = ['random']*12+['exp']*12+['dep']*12
nsms = ['random', 'exp', 'dep']
# models = ['kg4sl','kg4sl','kg4sl','kg4sl','kg4sl','kg4sl','kg4sl','kg4sl','kg4sl','kg4sl','kg4sl']
# models = ['kg4sl','kg4sl','kg4sl','kg4sl','kg4sl','kg4sl','kg4sl','kg4sl','kg4sl','kg4sl','kg4sl']
# models = ['sl2mf', 'slmgae', 'cmfw', 'ddgcn', 'gcatsl', 'grsmf', 'kg4sl', 'mge4sl', 'nsf4sl', 'pilsl', 'ptgnn']
models = ['sl2mf', 'slmgae', 'cmfw', 'ddgcn', 'gcatsl', 'grsmf', 'kg4sl', 'mge4sl', 'nsf4sl', 'pilsl', 'ptgnn','slgnn']
for i in range(36):
    score_values[i,0] = models[i%12]
    model = models[i%12]
    nsm = nsms[i//12]
    for j in range(2,182):
        cvx,pnr,met = all_col_name[j].split(',')
        r = cvx+','+pnr
        c = met
        try:
            score_values_df.loc[i,all_col_name[j]] = res_dict[nsm][model].loc[r,c]
            score_values_df.loc[i,'Model']=model
            score_values_df.loc[i,'NSM']=nsm
            
        except:
            # score_values_df.loc[i,all_col_name[j]] = res_dict[nsm][model].loc[r,c]
            score_values_df.loc[i,'Model']=model
            score_values_df.loc[i,'NSM']=nsm

In [ ]:
# new_df = pd.DataFrame(data = score_values,columns=all_col_name)
# new_df
score_values_df

In [ ]:
score_values_df.to_csv('summary_all_matrics_wo_compt.csv', index=False)

In [ ]:
score_values_df.fillna(np.nan,inplace=True)

In [ ]:
sub_score_values_df = score_values_df[list(score_values_df.columns[:2])+[c for c in score_values_df.columns if any(['F1' in c,'F1' in c,'F1' in c])]]
sub_score_values_df

In [ ]:
# cmap = sns.cubehelix_palette(as_cmap=True, nan_color='gray')
f, ax = plt.subplots(figsize=(10, 12))
sns.heatmap(sub_score_values_df.iloc[:,2:]-wo_sub_score_values_df.iloc[:,2:],annot=True,fmt='.3f',annot_kws={"size": 9},ax=ax,yticklabels=score_values_df['Model'],cbar=False)
# sns.heatmap(sub_score_values_df.iloc[:,2:],annot=True,fmt='.3f',annot_kws={"size": 9},ax=ax,yticklabels=score_values_df['Model'],cbar=False)
ax.tick_params(axis='both', which='major', labelsize=9)
# 将X轴刻度标签移动到顶部
ax.xaxis.tick_top()
# （可选）旋转X轴标签以改善可读性，这里设置为水平方向
ax.set_xticklabels(ax.get_xticklabels(), rotation=40)
plt.tight_layout()
# plt.savefig('mdamb231_gene_frequency_heatmap_top50.png')

In [ ]:
# cmap = sns.cubehelix_palette(as_cmap=True, nan_color='gray')
f, ax = plt.subplots(figsize=(10, 12))
sns.heatmap(sub_score_values_df.iloc[:,2:],
            annot=True,fmt='.3f',annot_kws={"size": 9},ax=ax,yticklabels=pd.read_csv('/home/yimiaofeng/MyProject/SL-Benchmark/src/summary_all_matrics.csv')['Model'],cbar=False)
ax.tick_params(axis='both', which='major', labelsize=9)
# 将X轴刻度标签移动到顶部
ax.xaxis.tick_top()
# （可选）旋转X轴标签以改善可读性，这里设置为水平方向
ax.set_xticklabels(ax.get_xticklabels(), rotation=40)
plt.tight_layout()
# plt.savefig('mdamb231_gene_frequency_heatmap_top50.png')

In [ ]:
res_dict['random'].keys()

# Compare w/wo compute labels

In [ ]:
summary_res = pd.read_csv('/home/yimiaofeng/MyProject/SL-Benchmark/src/summary_all_matrics.csv')
summary_res_wo_compt = pd.read_csv('/home/yimiaofeng/MyProject/SL-Benchmark/src/summary_all_matrics_wo_compt.csv')

In [ ]:
def extract_res(res,nsm):
    model_order = ['GRSMF', 'SL2MF', 'CMFW', 'SLMGAE', 'NSF4SL', 'PTGNN', 'PiLSL', 'KG4SL','SLGNN', 'DDGCN', 'GCATSL', 'MGE4SL']
    res = res[res['NSM']==nsm]
    res['Model'] = res['Model'].apply(lambda x: x.upper())
    res.index = res['Model']
    res = res.loc[[e.upper() for e in model_order]]
    res['Model'] = model_order
    return res

In [ ]:
random_res = extract_res(summary_res,'random')
random_res_wo_compt = extract_res(summary_res_wo_compt,'random')

exp_res = extract_res(summary_res,'exp')
exp_res_wo_compt = extract_res(summary_res_wo_compt,'exp')

dep_res = extract_res(summary_res,'dep')
dep_res_wo_compt = extract_res(summary_res_wo_compt,'dep')

In [ ]:
f1_random_res = random_res[sorted([c for c in random_res.columns if all(['F1' in c,'F1' in c,'F1' in c])])]
f1_exp_res = exp_res[sorted([c for c in exp_res.columns if all(['F1' in c,'F1' in c,'F1' in c])])]
f1_dep_res = dep_res[sorted([c for c in dep_res.columns if all(['F1' in c,'F1' in c,'F1' in c])])]
n10_random_res = random_res[sorted([c for c in random_res.columns if all(['N10' in c,'N10' in c,'N10' in c])])]
n10_exp_res = exp_res[sorted([c for c in exp_res.columns if all(['N10' in c,'N10' in c,'N10' in c])])]
n10_dep_res = dep_res[sorted([c for c in dep_res.columns if all(['N10' in c,'N10' in c,'N10' in c])])]

f1_random_res_wo_compt = random_res_wo_compt[sorted([c for c in random_res_wo_compt.columns if all(['F1' in c,'F1' in c,'F1' in c])])]
f1_exp_res_wo_compt = exp_res_wo_compt[sorted([c for c in exp_res_wo_compt.columns if all(['F1' in c,'F1' in c,'F1' in c])])]
f1_dep_res_wo_compt = dep_res_wo_compt[sorted([c for c in dep_res_wo_compt.columns if all(['F1' in c,'F1' in c,'F1' in c])])]
n10_random_res_wo_compt = random_res_wo_compt[sorted([c for c in random_res_wo_compt.columns if all(['N10' in c,'N10' in c,'N10' in c])])]
n10_exp_res_wo_compt = exp_res_wo_compt[sorted([c for c in exp_res_wo_compt.columns if all(['N10' in c,'N10' in c,'N10' in c])])]
n10_dep_res_wo_compt = dep_res_wo_compt[sorted([c for c in dep_res_wo_compt.columns if all(['N10' in c,'N10' in c,'N10' in c])])]

In [ ]:
def cal_classifi_score(class_res):
    cv1_res = class_res[[c for c in class_res.columns if 'CV1' in c]]
    cv2_res = class_res[[c for c in class_res.columns if 'CV2' in c]]
    cv3_res = class_res[[c for c in class_res.columns if 'CV3' in c]]
    class_score_df = pd.DataFrame()
    class_score_df['CV1'] = cv1_res.values[:,:9].mean(axis=1)
    class_score_df['CV2'] = cv2_res.values[:,:9].mean(axis=1)
    class_score_df['CV3'] = cv3_res.values[:,:9].mean(axis=1)
    class_score_df.index = class_res.index
    class_score_df['Weighted_123'] = class_score_df['CV1']*0.4+class_score_df['CV2']*0.5+class_score_df['CV3']*0.1
    class_score_df = class_score_df.sort_values(by='Weighted_123',ascending=False)
    class_score_df['Rank_123'] = list(range(1,13))
    class_score_df['Weighted_12'] = class_score_df['CV1']*0.4+class_score_df['CV2']*0.6
    class_score_df = class_score_df.sort_values(by='Weighted_12',ascending=False)
    class_score_df['Rank_12'] = list(range(1,13))
    class_score_df = class_score_df.loc[cv3_res.index.to_list()]
    return class_score_df

def cal_ranking_score(class_res):
    class_res.loc['PILSL'] = [np.nan]*class_res.shape[1]
    cv1_res = class_res[[c for c in class_res.columns if 'CV1' in c]]
    cv2_res = class_res[[c for c in class_res.columns if 'CV2' in c]]
    cv3_res = class_res[[c for c in class_res.columns if 'CV3' in c]]
    class_score_df = pd.DataFrame()
    class_score_df['CV1'] = cv1_res.values[:,:9].mean(axis=1)
    class_score_df['CV2'] = cv2_res.values[:,:9].mean(axis=1)
    class_score_df['CV3'] = cv3_res.values[:,:9].mean(axis=1)
    class_score_df.index = class_res.index
    class_score_df['Weighted_123'] = class_score_df['CV1']*0.4+class_score_df['CV2']*0.5+class_score_df['CV3']*0.1
    class_score_df = class_score_df.sort_values(by='Weighted_123',ascending=False)
    class_score_df['Rank_123'] = list(range(1,13))
    class_score_df['Weighted_12'] = class_score_df['CV1']*0.4+class_score_df['CV2']*0.6
    class_score_df = class_score_df.sort_values(by='Weighted_12',ascending=False)
    class_score_df['Rank_12'] = list(range(1,13))
    class_score_df = class_score_df.loc[cv3_res.index.to_list()]
    return class_score_df

In [ ]:
def cal_overall_score(class_res,rank_res):
    over_all_123 = pd.DataFrame((class_res['Weighted_123'].values+rank_res['Weighted_123'].values)/2,columns=['Weighted_123'])
    over_all_12 = pd.DataFrame((class_res['Weighted_12'].values+rank_res['Weighted_12'].values)/2,columns=['Weighted_12'])
    overall = pd.concat([over_all_123,over_all_12],axis=1)
    overall.index = class_res.index
    overall.loc['PILSL'] = [class_res['Weighted_123'].values[6]/2,class_res['Weighted_12'].values[6]/2]
    overall = overall.sort_values(by='Weighted_123',ascending=False)
    overall['Rank_123'] = list(range(1,13))
    overall = overall.sort_values(by='Weighted_12',ascending=False)
    overall['Rank_12'] = list(range(1,13))
    overall = overall.loc[class_res.index.to_list()]
    return overall

In [ ]:
class_random_res = random_res[sorted([c for c in random_res.columns if any(['AUROC' in c,'AUPR' in c,'F1' in c])])]
class_exp_res = exp_res[sorted([c for c in exp_res.columns if any(['AUROC' in c,'AUPR' in c,'F1' in c])])]
class_dep_res = dep_res[sorted([c for c in dep_res.columns if any(['AUROC' in c,'AUPR' in c,'F1' in c])])]
rank_random_res = random_res[sorted([c for c in random_res.columns if any(['N10' in c,'R10' in c,'P10' in c])])]
rank_exp_res = exp_res[sorted([c for c in exp_res.columns if any(['N10' in c,'R10' in c,'P10' in c])])]
rank_dep_res = dep_res[sorted([c for c in dep_res.columns if any(['N10' in c,'R10' in c,'P10' in c])])]

class_random_res_wo_compt = random_res_wo_compt[sorted([c for c in random_res_wo_compt.columns if any(['AUROC' in c,'AUPR' in c,'F1' in c])])]
class_exp_res_wo_compt = exp_res_wo_compt[sorted([c for c in exp_res_wo_compt.columns if any(['AUROC' in c,'AUPR' in c,'F1' in c])])]
class_dep_res_wo_compt = dep_res_wo_compt[sorted([c for c in dep_res_wo_compt.columns if any(['AUROC' in c,'AUPR' in c,'F1' in c])])]
rank_random_res_wo_compt = random_res_wo_compt[sorted([c for c in random_res_wo_compt.columns if any(['N10' in c,'R10' in c,'P10' in c])])]
rank_exp_res_wo_compt = exp_res_wo_compt[sorted([c for c in exp_res_wo_compt.columns if any(['N10' in c,'R10' in c,'P10' in c])])]
rank_dep_res_wo_compt = dep_res_wo_compt[sorted([c for c in dep_res_wo_compt.columns if any(['N10' in c,'R10' in c,'P10' in c])])]

class_score_random = cal_classifi_score(class_random_res)
class_score_exp = cal_classifi_score(class_exp_res)
class_score_dep = cal_classifi_score(class_dep_res)
class_score_random_wo_compt = cal_classifi_score(class_random_res_wo_compt)
class_score_exp_wo_compt = cal_classifi_score(class_exp_res_wo_compt)
class_score_dep_wo_compt = cal_classifi_score(class_dep_res_wo_compt)

rank_score_random = cal_ranking_score(rank_random_res)
rank_score_exp = cal_ranking_score(rank_exp_res)
rank_score_dep = cal_ranking_score(rank_dep_res)
rank_score_random_wo_compt = cal_ranking_score(rank_random_res_wo_compt)
rank_score_exp_wo_compt = cal_ranking_score(rank_exp_res_wo_compt)
rank_score_dep_wo_compt = cal_ranking_score(rank_dep_res_wo_compt)

In [ ]:
rank_score_random

In [ ]:
rank_score_random_wo_compt

In [ ]:
cal_overall_score(class_score_random,rank_score_random)

In [ ]:
cal_overall_score(class_score_dep,rank_score_dep)

In [ ]:
cal_overall_score(class_score_exp,rank_score_exp)

In [ ]:
rank_random_res_wo_compt[['CV1,1,N10','CV1,5,N10','CV1,20,N10','CV1,50,N10']].round(3).to_csv('tmp.csv')

In [ ]:
(class_score_dep['Weighted_123']+rank_score_dep['Weighted_123'])/2

In [ ]:
(class_random_res[['CV3,1,F1','CV3,5,F1','CV3,20,F1','CV3,50,F1']].round(3)).to_csv('./tmp.csv')
# (class_random_res[['CV1,1,AUROC','CV1,1,AUPR','CV1,1,F1','CV1,5,AUROC','CV1,5,AUPR','CV1,5,F1','CV1,20,AUROC','CV1,20,AUPR','CV1,20,F1','CV1,50,AUROC','CV1,50,AUPR','CV1,50,F1']].round(3)*100).T.to_csv('./tmp.csv')
# class_exp_res_wo_compt[['CV1,1,AUROC','CV1,5,AUROC','CV1,20,AUROC','CV1,50,AUROC']]
# class_exp_res_wo_compt[['CV1,1,F1','CV1,5,F1','CV1,20,F1','CV1,50,F1']]

In [ ]:
class_random_res[['CV2,1,F1','CV2,5,F1','CV2,20,F1','CV2,50,F1']]

In [ ]:
class_score_random = class_score_random.rename(columns={'CV1':'CV1_random','CV2':'CV2_random','CV3':'CV3_random','Weighted_123':'Weighted_123_random','Rank_123':'Rank_123_random','Weighted_12':'Weighted_12_random','Rank_12':'Rank_12_random'})
class_score_exp = class_score_exp.rename(columns={'CV1':'CV1_exp','CV2':'CV2_exp','CV3':'CV3_exp','Weighted_123':'Weighted_123_exp','Rank_123':'Rank_123_exp','Weighted_12':'Weighted_12_exp','Rank_12':'Rank_12_exp'})
class_score_dep = class_score_dep.rename(columns={'CV1':'CV1_dep','CV2':'CV2_dep','CV3':'CV3_dep','Weighted_123':'Weighted_123_dep','Rank_123':'Rank_123_dep','Weighted_12':'Weighted_12_dep','Rank_12':'Rank_12_dep'})
all_class_score = pd.concat([class_score_random,class_score_exp,class_score_dep],axis=1)
all_class_score = all_class_score[['Weighted_123_random','Weighted_123_exp','Weighted_123_dep']]
all_class_score = all_class_score.rename(columns={'Weighted_123_random':'Random','Weighted_123_exp':'Exp','Weighted_123_dep':'Dep'})

class_score_random_wo_compt = class_score_random_wo_compt.rename(columns={'CV1':'CV1_random','CV2':'CV2_random','CV3':'CV3_random','Weighted_123':'Weighted_123_random','Rank_123':'Rank_123_random','Weighted_12':'Weighted_12_random','Rank_12':'Rank_12_random'})
class_score_exp_wo_compt = class_score_exp_wo_compt.rename(columns={'CV1':'CV1_exp','CV2':'CV2_exp','CV3':'CV3_exp','Weighted_123':'Weighted_123_exp','Rank_123':'Rank_123_exp','Weighted_12':'Weighted_12_exp','Rank_12':'Rank_12_exp'})
class_score_dep_wo_compt = class_score_dep_wo_compt.rename(columns={'CV1':'CV1_dep','CV2':'CV2_dep','CV3':'CV3_dep','Weighted_123':'Weighted_123_dep','Rank_123':'Rank_123_dep','Weighted_12':'Weighted_12_dep','Rank_12':'Rank_12_dep'})
all_class_score_wo_compt = pd.concat([class_score_random_wo_compt,class_score_exp_wo_compt,class_score_dep_wo_compt],axis=1)
all_class_score_wo_compt = all_class_score_wo_compt[['Weighted_123_random','Weighted_123_exp','Weighted_123_dep']]
all_class_score_wo_compt = all_class_score_wo_compt.rename(columns={'Weighted_123_random':'Random','Weighted_123_exp':'Exp','Weighted_123_dep':'Dep'})

rank_score_random = rank_score_random.rename(columns={'CV1':'CV1_random','CV2':'CV2_random','CV3':'CV3_random','Weighted_123':'Weighted_123_random','Rank_123':'Rank_123_random','Weighted_12':'Weighted_12_random','Rank_12':'Rank_12_random'})
rank_score_exp = rank_score_exp.rename(columns={'CV1':'CV1_exp','CV2':'CV2_exp','CV3':'CV3_exp','Weighted_123':'Weighted_123_exp','Rank_123':'Rank_123_exp','Weighted_12':'Weighted_12_exp','Rank_12':'Rank_12_exp'})
rank_score_dep = rank_score_dep.rename(columns={'CV1':'CV1_dep','CV2':'CV2_dep','CV3':'CV3_dep','Weighted_123':'Weighted_123_dep','Rank_123':'Rank_123_dep','Weighted_12':'Weighted_12_dep','Rank_12':'Rank_12_dep'})
all_rank_score = pd.concat([rank_score_random,rank_score_exp,rank_score_dep],axis=1)
all_rank_score = all_rank_score[['Weighted_123_random','Weighted_123_exp','Weighted_123_dep']]
all_rank_score = all_rank_score.rename(columns={'Weighted_123_random':'Random','Weighted_123_exp':'Exp','Weighted_123_dep':'Dep'})

rank_score_random_wo_compt = rank_score_random_wo_compt.rename(columns={'CV1':'CV1_random','CV2':'CV2_random','CV3':'CV3_random','Weighted_123':'Weighted_123_random','Rank_123':'Rank_123_random','Weighted_12':'Weighted_12_random','Rank_12':'Rank_12_random'})
rank_score_exp_wo_compt = rank_score_exp_wo_compt.rename(columns={'CV1':'CV1_exp','CV2':'CV2_exp','CV3':'CV3_exp','Weighted_123':'Weighted_123_exp','Rank_123':'Rank_123_exp','Weighted_12':'Weighted_12_exp','Rank_12':'Rank_12_exp'})
rank_score_dep_wo_compt = rank_score_dep_wo_compt.rename(columns={'CV1':'CV1_dep','CV2':'CV2_dep','CV3':'CV3_dep','Weighted_123':'Weighted_123_dep','Rank_123':'Rank_123_dep','Weighted_12':'Weighted_12_dep','Rank_12':'Rank_12_dep'})
all_rank_score_wo_compt = pd.concat([rank_score_random_wo_compt,rank_score_exp_wo_compt,rank_score_dep_wo_compt],axis=1)
all_rank_score_wo_compt = all_rank_score_wo_compt[['Weighted_123_random','Weighted_123_exp','Weighted_123_dep']]
all_rank_score_wo_compt = all_rank_score_wo_compt.rename(columns={'Weighted_123_random':'Random','Weighted_123_exp':'Exp','Weighted_123_dep':'Dep'})

all_class_score_wo_compt

In [ ]:
class_score_random

In [ ]:
plot_df_c = all_class_score.melt(var_name='NSM', value_name='Score', value_vars=['Random','Exp','Dep'], ignore_index=False)
plot_df_c['Model'] = plot_df_c.index
plot_df_c['Task'] = 'Classification'
plot_df_c_wo_compt = all_class_score_wo_compt.melt(var_name='NSM', value_name='Score', value_vars=['Random','Exp','Dep'], ignore_index=False)
plot_df_c_wo_compt['Model'] = plot_df_c_wo_compt.index
plot_df_c_wo_compt['Task'] = 'Classification'

plot_df_r = all_rank_score.melt(var_name='NSM', value_name='Score', value_vars=['Random','Exp','Dep'], ignore_index=False)
plot_df_r['Model'] = plot_df_r.index
plot_df_r['Task'] = 'Ranking'
plot_df_r_wo_compt = all_rank_score_wo_compt.melt(var_name='NSM', value_name='Score', value_vars=['Random','Exp','Dep'], ignore_index=False)
plot_df_r_wo_compt['Model'] = plot_df_r_wo_compt.index
plot_df_r_wo_compt['Task'] = 'Ranking'

# plot_df = pd.concat([plot_df_c,plot_df_r],axis=0)
# plot_df

In [ ]:
plot_df_c

In [ ]:
sns.set_theme(style="whitegrid")
models = ['GRSMF','SL$^2$MF','CMFW','SLMGAE','NSF4SL','PTGNN','PiLSL','KG4SL','SLGNN','DDGCN','GCATSL','MGE4SL']*3
plot_df_c['Model'] = models
f, ax = plt.subplots(figsize=(5, 5))
sns.barplot(data=plot_df_c,y='Model',x='Score',hue='NSM',palette=['#F3A48A','#6FB0DE','#ABDE88'],alpha=0.9)
# sns.barplot(data=plot_df_c_wo_compt,y='Model',x='Score',hue='NSM',alpha=0.5)
ax.set(title='Comparison of classification scores',ylabel="",xlabel="Classification scores",xlim=(0,1))
plt.xlim(0,1.25)
plt.tight_layout()
plt.savefig(f'./fig/class_scores.svg',format='svg', bbox_inches='tight')

In [ ]:
sns.set_theme(style="whitegrid")
models = ['GRSMF','SL$^2$MF','CMFW','SLMGAE','NSF4SL','PTGNN','PiLSL','KG4SL','SLGNN','DDGCN','GCATSL','MGE4SL']*3
plot_df_r['Model'] = models
f, ax = plt.subplots(figsize=(5, 5))
sns.barplot(data=plot_df_r,y='Model',x='Score',hue='NSM',palette=['#F3A48A','#6FB0DE','#ABDE88'],alpha=0.9)
# sns.barplot(data=plot_df_c_wo_compt,y='Model',x='Score',hue='NSM',alpha=0.5)
ax.set(title='Comparison of ranking scores',ylabel="",xlabel="Ranking scores",xlim=(0,1))
plt.xlim(0,0.4)
plt.tight_layout()
plt.savefig(f'./fig/ranking_scores.svg',format='svg', bbox_inches='tight')

In [ ]:
# class_score_random_wo_compt
# rank_score_random_wo_compt
class_score_random

In [ ]:
cvalues = class_random_res_wo_compt[['CV1,1,F1','CV2,1,F1','CV3,1,F1']].values
rvalues = n10_random_res_wo_compt[['CV1,1,N10','CV2,1,N10','CV3,1,N10']].values
for i in range(12):
    s = '&'.join([str(e.round(3)) for e in list(cvalues[i])+list(rvalues[i])])
    print(f'{i+1} & {s} \\\\')

In [ ]:
n10_random_res_wo_compt[['CV1,1,N10','CV1,5,N10','CV1,20,N10','CV1,50,N10']]

In [ ]:
# class_random_res_wo_compt[['CV1,1,F1','CV1,5,F1','CV1,20,F1','CV1,50,F1']]-class_random_res[['CV1,1,F1','CV1,5,F1','CV1,20,F1','CV1,50,F1']]
class_random_res_wo_compt[['CV1,1,F1','CV2,1,F1','CV3,1,F1']]-class_random_res[['CV1,1,F1','CV2,1,F1','CV3,1,F1']]

In [ ]:
def diff_plot(f1_exp_res,f1_random_res,task,title,output_dir):
    if task=='class':
        senario_order = ['CV1,1,F1','CV1,5,F1','CV1,20,F1','CV1,50,F1','CV2,1,F1','CV2,5,F1','CV2,20,F1','CV2,50,F1','CV3,1,F1','CV3,5,F1','CV3,20,F1','CV3,50,F1']
    elif task=='rank':
        senario_order = ['CV1,1,N10','CV1,5,N10','CV1,20,N10','CV1,50,N10','CV2,1,N10','CV2,5,N10','CV2,20,N10','CV2,50,N10','CV3,1,N10','CV3,5,N10','CV3,20,N10','CV3,50,N10']
    models = ['GRSMF','SL$^2$MF','CMFW','SLMGAE','NSF4SL','PTGNN','PiLSL','KG4SL','SLGNN','DDGCN','GCATSL','MGE4SL']
    sns.set_theme(style="white")

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(6, 5))
    diff_score = f1_exp_res[senario_order]-f1_random_res[senario_order]
    diff_score.index = models
    if task=='rank':
        diff_score.loc['PiLSL']=[np.nan]*diff_score.shape[1]
    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(diff_score,cmap=sns.color_palette('vlag', as_cmap=True), center=0,
                square=True, linewidths=.5)

    ax.set(title=title,
        ylabel="",)
    plt.xticks(rotation=36)
    plt.savefig(f'./fig/{output_dir}.svg',format='svg', bbox_inches='tight')

In [ ]:
diff_plot(f1_exp_res,f1_random_res,'class','F1 score difference between Exp and Random','f1_diff_exp')
diff_plot(f1_dep_res,f1_random_res,'class','F1 score difference between Dep and Random','f1_diff_dep')
diff_plot(f1_exp_res_wo_compt,f1_random_res_wo_compt,'class','F1 score difference between Exp and Random','f1_diff_exp_wo_compt')
diff_plot(f1_dep_res_wo_compt,f1_random_res_wo_compt,'class','F1 score difference between Dep and Random','f1_diff_dep_wo_compt')

diff_plot(n10_exp_res,n10_random_res,'rank','NDCG@10 score difference between Exp and Random','n10_diff_exp')
diff_plot(n10_dep_res,n10_random_res,'rank','NDCG@10 score difference between Dep and Random','n10_diff_dep')
diff_plot(n10_exp_res_wo_compt,n10_random_res_wo_compt,'rank','NDCG@10 score difference between Exp and Random','n10_diff_exp_wo_compt')
diff_plot(n10_dep_res_wo_compt,n10_random_res_wo_compt,'rank','NDCG@10 score difference between Dep and Random','n10_diff_dep_wo_compt')


# Statistic of GO PPI etc.

In [ ]:
gene2go = pd.read_csv('/data/yimiaofeng/SLBench/scripts/orifiles/gene2go',sep='\t')
gene2go = gene2go[gene2go['#tax_id']==9606]
meta_table_9845 = pd.read_csv('/home/yimiaofeng/MyProject/SL-Benchmark/data/preprocessed_data/meta_table_9845.csv')
# goa_human = pd.read_csv('/home/yimiaofeng/MyProject/SL-Benchmark/src/goa_human.gaf',sep='\t',header=None,skiprows=13)
# goa_human=goa_human[goa_human[2].isin(set(meta_table_9845['symbol']))]
# goa_human=goa_human[goa_human[8]=='P']
gene2go = gene2go[gene2go['GeneID'].isin(set(meta_table_9845['entrez_id']))]
gene2go = gene2go[gene2go['Category']=='Component']
gene2go

In [ ]:
KEGG_pathway_hsa_uniq = pd.read_csv('/data/yimiaofeng/SLBench/scripts/KEGG_pathway_hsa_uniq.txt',sep='|')
KEGG_pathway_hsa_uniq = KEGG_pathway_hsa_uniq[KEGG_pathway_hsa_uniq['ko_name'].isin(set(meta_table_9845['symbol']))]
len(set(KEGG_pathway_hsa_uniq['level3_pathway_id']))
# KEGG_pathway_hsa_uniq

# Score

In [ ]:
human_sl_6460 = pd.read_csv('/home/yimiaofeng/MyProject/SL-Benchmark/data/preprocessed_data/human_sl_6460.csv',index_col=0)
human_sl_9845 = pd.read_csv('/home/yimiaofeng/MyProject/SL-Benchmark/data/preprocessed_data/human_sl_9845.csv')
meta_table_9845 = pd.read_csv('/home/yimiaofeng/MyProject/SL-Benchmark/data/preprocessed_data/meta_table_9845.csv')
all_uni_id_cut = np.load('/home/yimiaofeng/MyProject/SL-Benchmark/data/preprocessed_data/wo_compt/cut_all_uni_id.npy',allow_pickle=True).item()
human_sl_6460['unified_id_A'] = human_sl_6460['unified_id_A'].map(all_uni_id_cut)
human_sl_6460['unified_id_B'] = human_sl_6460['unified_id_B'].map(all_uni_id_cut)
# human_sl_6460 = human_sl_6460[human_sl_6460['unified_id_A']<human_sl_6460['unified_id_B']]
# human_sl_9845 = human_sl_9845[human_sl_9845['unified_id_A']<human_sl_9845['unified_id_B']]
sp_9845 = sp.csr_matrix((np.ones(len(human_sl_9845)), (human_sl_9845['unified_id_A'], human_sl_9845['unified_id_B'])), shape=(9845,9845),dtype=bool)
sp_9845 = sp_9845+sp_9845.T
sp_6460 = sp.csr_matrix((np.ones(len(human_sl_6460)), (human_sl_6460['unified_id_A'], human_sl_6460['unified_id_B'])), shape=(9845,9845),dtype=bool)
sp_6460 = sp_6460+sp_6460.T
sp_comp = sp_9845-sp_6460
comp_sl = pd.DataFrame({'unified_id_A':sp_comp.nonzero()[0],'unified_id_B':sp_comp.nonzero()[1]})
comp_sl = comp_sl[comp_sl['unified_id_A']<comp_sl['unified_id_B']]
comp_sl
# human_sl_6460
# human_sl_9845
# all_uni_id_cut

In [ ]:
DepMap_Effect_corr = pd.read_csv('/home/yimiaofeng/MyProject/SL-Benchmark/data/preprocessed_data/DepMap_Effect_corr.csv',index_col=0)
DepMap_Expression_corr = pd.read_csv('/home/yimiaofeng/MyProject/SL-Benchmark/data/preprocessed_data/DepMap_Expression_corr.csv',index_col=0)
DepMap_Effect_corr.columns = DepMap_Effect_corr.columns.map(int)
DepMap_Effect_corr.index = DepMap_Effect_corr.index.map(int)
DepMap_Expression_corr.columns = DepMap_Expression_corr.columns.map(int)
DepMap_Expression_corr.index = DepMap_Expression_corr.index.map(int)
idx = []
for i in DepMap_Expression_corr.index:
    if all(DepMap_Expression_corr.loc[i,:]==1):
        idx.append(i)
for i in idx:
    DepMap_Expression_corr.loc[i,:]=[np.nan]*len(DepMap_Expression_corr.columns)
    DepMap_Expression_corr.loc[:,i]=[np.nan]*len(DepMap_Expression_corr.columns)
DepMap_Expression_corr = DepMap_Expression_corr[list(range(9845))]
DepMap_Expression_corr = DepMap_Expression_corr.sort_index()
DepMap_Expression_corr

In [ ]:
def ex_score(human_sl_6460,DepMap_Expression_corr):
    score = []
    for row in human_sl_6460.values:
        score.append(DepMap_Expression_corr.loc[int(row[0]),int(row[1])])
    return score
human_sl_6460_exp = ex_score(human_sl_6460,DepMap_Expression_corr)
human_sl_6460_dep = ex_score(human_sl_6460,DepMap_Effect_corr)
human_sl_9845_exp = ex_score(human_sl_9845,DepMap_Expression_corr)
human_sl_9845_dep = ex_score(human_sl_9845,DepMap_Effect_corr)
comp_sl_exp = ex_score(comp_sl,DepMap_Expression_corr)
comp_sl_dep = ex_score(comp_sl,DepMap_Effect_corr)

In [ ]:
f,ax = plt.subplots(figsize=(5, 5))

# Plot a histogram of the correlation
sns.kdeplot(data=human_sl_6460_exp, fill=True,label='Dataset w/o computational labels',color='#F3A48A', common_norm = True, linewidth=0.5,alpha=.5)
sns.kdeplot(data=human_sl_9845_exp, fill=True,label='Complete dataset',color='#6FB0DE', common_norm = True, linewidth=0.5,alpha=.5)
sns.kdeplot(data=comp_sl_exp, fill=True,label='Computational labels',color='#ABDE88', common_norm = True, linewidth=0.5,alpha=.5)
ax.legend(ncol=1, loc="upper left", frameon=True)
# plt.hist(sl_corr, bins=100)
plt.xlabel('Correlation coefficient')
plt.ylabel('Frequency')
plt.title('Distribution of gene expression correlation coefficients \n from different source labels')
plt.xlim(-1,1)
plt.ylim(0,3.3)
# plt.show()
plt.savefig('./fig/all_part_compt.svg',format='svg', bbox_inches='tight')

In [ ]:
f,ax = plt.subplots(figsize=(5, 5))

# Plot a histogram of the correlation
sns.kdeplot(data=human_sl_9845_exp, fill=True,label='Gene dependency score',color='#F3A48A', common_norm = True, linewidth=0.5,alpha=.5)
sns.kdeplot(data=human_sl_9845_dep, fill=True,label='Gene expression',color='#6FB0DE', common_norm = True, linewidth=0.5,alpha=.5)
# sns.kdeplot(data=comp_sl_exp, fill=True,label='Computational labels',color='#ABDE88', common_norm = True, linewidth=0.5,alpha=.5)
ax.legend(ncol=1, loc="upper left", frameon=True)
# plt.hist(sl_corr, bins=100)
plt.xlabel('Correlation coefficient')
plt.ylabel('Frequency')
plt.title('Distribution of correlation coefficients \n calculated using different data')
plt.xlim(-1,1)
plt.ylim(0,7.6)
# plt.show()
plt.savefig('./fig/exp_dep.svg',format='svg', bbox_inches='tight')

In [ ]:
f,ax = plt.subplots(figsize=(7.5, 7.5))

# Plot a histogram of the correlation
sns.kdeplot(data=human_sl_6460_dep, fill=True,label='Dataset w/o computational labels', common_norm = True, linewidth=0.5,alpha=.8)
sns.kdeplot(data=human_sl_9845_dep, fill=True,label='Complete dataset', common_norm = True, linewidth=0.5,alpha=.8)
sns.kdeplot(data=comp_sl_dep, fill=True,label='Computational labels', common_norm = True, linewidth=0.5,alpha=.8)

# plt.hist(sl_corr, bins=100)
plt.xlabel('Gene dependency score correlation coefficient')
plt.ylabel('Frequency')
plt.title('Distribution of known SL gene pairs')
plt.xlim(-1,1)
# plt.show()
# plt.savefig('sl_corr_exp_18.svg')

# Score distribution

#### Note: the following code is used to generate the score distribution figure for the paper
#### **Must use parameter --save_mat to successfully run the following code**

In [ ]:
nsm = 'random'
base_path = f'../results/{nsm}_score_mats/'
models = os.listdir(base_path)
res_names = os.listdir(f'{base_path}/{models[0]}')
res_names_classify = [i for i in res_names if 'classify' in i]
res_names_ranking = [i for i in res_names if 'ranking' in i]


In [ ]:
pos_samples, neg_samples = np.load('../data/data_split/CV1_1.npy',allow_pickle=True)
_, _, train_pos_kfold, test_pos_kfold = pos_samples
_, _, train_neg_kfold, test_neg_kfold = neg_samples

In [ ]:
train_pos_kfold[0].shape

In [ ]:
cvx = ['CV1', 'CV2', 'CV3']
pnrs = [1.0,0.2,0.05,0.02]
train_pos_scores = []
train_neg_scores = []
test_pos_scores =[]
test_neg_scores =[]
for fold_num in range(5):

    score_mat = np.load(f'{base_path}/{models[0]}/slgnn_fold_{fold_num}_pos_neg_1.0_CV1_Random_ranking.npy')
    train_pos_scores.append(score_mat[train_pos_kfold[fold_num][:,0],train_pos_kfold[fold_num][:,1]])
    train_neg_scores.append(score_mat[train_neg_kfold[fold_num][:,0],train_neg_kfold[fold_num][:,1]])
    test_pos_scores.append(score_mat[test_pos_kfold[fold_num][:,0],test_pos_kfold[fold_num][:,1]])
    test_neg_scores.append(score_mat[test_neg_kfold[fold_num][:,0],test_neg_kfold[fold_num][:,1]])

In [ ]:
plt.close('all')

In [ ]:
def except_abnormal(data):
    Q1 = np.percentile(data, 5)
    Q3 = np.percentile(data, 95)
    IQR = Q3 - Q1
    filtered_data = data[(data >= Q1) & (data <= Q3)]
    return filtered_data


In [ ]:
def extract_data(cv,pnr,fold_num,base_path,model):
    pos_samples, neg_samples = np.load(f'../data/data_split/{cv}_{int(1/float(pnr))}.npy',allow_pickle=True)
    _, _, train_pos_kfold, test_pos_kfold = pos_samples
    _, _, train_neg_kfold, test_neg_kfold = neg_samples
    
    score_mat = np.load(f'{base_path}/{model}/{model}_fold_{fold_num}_pos_neg_{pnr}_{cv}_Random_classify.npy')

    train_pos_score = except_abnormal(score_mat[train_pos_kfold[fold_num][:,0],train_pos_kfold[fold_num][:,1]])
    train_neg_score = except_abnormal(score_mat[train_neg_kfold[fold_num][:,0],train_neg_kfold[fold_num][:,1]])
    test_pos_score = except_abnormal(score_mat[test_pos_kfold[fold_num][:,0],test_pos_kfold[fold_num][:,1]])
    test_neg_score = except_abnormal(score_mat[test_neg_kfold[fold_num][:,0],test_neg_kfold[fold_num][:,1]])
        
    return train_pos_score, train_neg_score, test_pos_score, test_neg_score

In [ ]:
def load_score_data(train_pos_score, train_neg_score, test_pos_score, test_neg_score):

    plot_df = pd.DataFrame(columns=['Predict score','Sample type'])
    plot_df = plot_df.append(pd.DataFrame({'Predict score':train_pos_score,'Sample type':['$Train_{pos}$']*len(train_pos_score)}))
    plot_df = plot_df.append(pd.DataFrame({'Predict score':train_neg_score,'Sample type':['$Train_{neg}$']*len(train_neg_score)}))
    plot_df = plot_df.append(pd.DataFrame({'Predict score':test_pos_score,'Sample type':['$Test_{pos}$']*len(test_pos_score)}))
    plot_df = plot_df.append(pd.DataFrame({'Predict score':test_neg_score,'Sample type':['$Test_{neg}$']*len(test_neg_score)}))
    return plot_df

In [ ]:
def normalize_data(all_score):
    biggest_score = -np.inf
    smallest_score = np.inf
    for k in all_score.keys():
        train_pos_score, train_neg_score, test_pos_score, test_neg_score = all_score[k]
        max_score = max(train_pos_score.max(),train_neg_score.max(),test_pos_score.max(),test_neg_score.max())
        min_score = min(train_pos_score.min(),train_neg_score.min(),test_pos_score.min(),test_neg_score.min())
        if max_score>biggest_score:
            biggest_score = max_score
        if min_score<smallest_score:
            smallest_score = min_score
    new_all_score = {}
    for k in all_score.keys():
        train_pos_score, train_neg_score, test_pos_score, test_neg_score = all_score[k]
        train_pos_score = (train_pos_score-smallest_score)/(biggest_score-smallest_score)
        train_neg_score = (train_neg_score-smallest_score)/(biggest_score-smallest_score)
        test_pos_score = (test_pos_score-smallest_score)/(biggest_score-smallest_score)
        test_neg_score = (test_neg_score-smallest_score)/(biggest_score-smallest_score)
        new_all_score[k] = train_pos_score, train_neg_score, test_pos_score, test_neg_score
    return new_all_score

In [ ]:
def prepare_data(fold_num,base_path,model):
    processed_data = {}
    for scenario in ['CV1_1.0','CV2_1.0','CV3_1.0','CV1_0.2','CV1_0.05','CV1_0.02']:
        cv, pnr = scenario.split('_')
        processed_data[scenario] = extract_data(cv,pnr,fold_num,base_path,model)
    processed_data = normalize_data(processed_data)
    processed_df = {}
    for k in processed_data.keys():
        train_pos_score, train_neg_score, test_pos_score, test_neg_score = processed_data[k]
        processed_df[k] = load_score_data(train_pos_score, train_neg_score, test_pos_score, test_neg_score)
    return processed_df

In [ ]:
nsm = 'random'
base_path = f'../results/{nsm}_score_mats/'
models = ['slgnn']
# models = ['gcatsl','slmgae','sl2mf','cmfw','ddgcn','grsmf','kg4sl','mge4sl','nsf4sl','ptgnn']
# fig, axes = plt.subplots(1, 6, figsize=(18, 3))
# fig_pos = [[0,0],[0,3],[0,4],[0,5],[0,1],[0,2]]
fig_pos = [[0,0],[0,1],[0,2],[1,0],[1,1],[1,2]]

for fold_num in range(5):
    for model in models:
        fig, axes = plt.subplots(2, 3, figsize=(12, 8))
        fig_num = 0
        # fold_num = 4
        plot_legend = False
        processed_df = prepare_data(fold_num,base_path,model)
        for k in processed_df.keys():
            cv, pnr = k.split('_')
            sns.kdeplot(data=processed_df[k], x='Predict score', hue='Sample type', fill=True, common_norm=False, alpha=.4, 
                        linewidth=0.5, ax=axes[fig_pos[fig_num][0],fig_pos[fig_num][1]], legend=plot_legend)
            axes[fig_pos[fig_num][0],fig_pos[fig_num][1]].set_title(f'SLGNN ({cv}, 1:{int(1/float(pnr))})')
            axes[fig_pos[fig_num][0],fig_pos[fig_num][1]].set_xlim(-0.1, 1.1)
            fig_num +=1
        # plt.savefig(f"../results/score_dist/{model}_{fold_num}_Random.svg", bbox_inches='tight')
        plt.savefig(f"../results/score_dist/{model}_{fold_num}_Random.svg",format='svg', bbox_inches='tight')

In [ ]:
cvx = ['CV1', 'CV2', 'CV3']
pnrs = [1.0,0.2,0.05,0.02]
nsm = 'random'
base_path = f'../results/{nsm}_score_mat/'
models = ['gcatsl','slmgae','sl2mf','cmfw','ddgcn','grsmf','kg4sl','mge4sl','nsf4sl','ptgnn']
# fig_pos = [[0,0],[0,3],[0,4],[0,5],[0,1],[0,2]]
fig_pos = [[0,0],[1,0],[1,1],[1,2],[0,1],[0,2]]

for fold_num in range(5):
    for model in models:
        fig, axes = plt.subplots(2, 3, figsize=(12, 8))
        fig_num = 0
        # fold_num = 4
        plot_legend = False
        for cv in cvx:
            if cv == 'CV1':
                for pnr in pnrs:
                    
                    plot_df = load_score_data(cv,pnr,fold_num,base_path,model)
                    
                    sns.kdeplot(data=plot_df, x='Predict score', hue='Sample type', fill=True, common_norm=False, alpha=.4, 
                                linewidth=0.5, ax=axes[fig_pos[fig_num][0],fig_pos[fig_num][1]], legend=plot_legend)
                    axes[fig_pos[fig_num][0],fig_pos[fig_num][1]].set_title(f'{model.upper()} ({cv}, 1:{int(1/float(pnr))})')
                    axes[fig_pos[fig_num][0],fig_pos[fig_num][1]].set_xlim(-0.1, 1.1)
                    
                    # # 获取当前kdeplot的legend数据，以便后续使用
                    # if legend_data is None:
                    #     legend_data = [line for line in ax.lines if line.get_label() != "_no_legend_"]
                    #     legend_labels = [line.get_label() for line in legend_data]

                    fig_num +=1
            else:
                if fig_num == 5:
                    plot_legend = True
                pnr = 1.0
                plot_df = load_score_data(cv,pnr,fold_num,base_path,model)

                sns.kdeplot(data=plot_df, x='Predict score', hue='Sample type', fill=True, common_norm=False, alpha=.4, 
                            linewidth=0.5, ax=axes[fig_pos[fig_num][0],fig_pos[fig_num][1]], legend=plot_legend)
                axes[fig_pos[fig_num][0],fig_pos[fig_num][1]].set_title(f'{model.upper()} ({cv}, 1:{int(1/float(pnr))})')
                axes[fig_pos[fig_num][0],fig_pos[fig_num][1]].set_xlim(-0.1, 1.1)

                fig_num +=1

        # plt.tight_layout()
        # plt.show()
        plt.savefig(f"../results/score_dist/{model}_{fold_num}_Random.svg",format='svg', bbox_inches='tight')

In [ ]:
plt.close('all')

In [ ]:
cv = 'CV1'
pnr = 1.0
model = 'ptgnn'
pos_samples, neg_samples = np.load(f'../data/data_split/{cv}_{int(1/float(pnr))}.npy',allow_pickle=True)
_, _, train_pos_kfold, test_pos_kfold = pos_samples
_, _, train_neg_kfold, test_neg_kfold = neg_samples

score_mat = np.load(f'{base_path}{model}/{model}_fold_{fold_num}_pos_neg_{pnr}_{cv}_Random_classify.npy')
train_pos_score = score_mat[train_pos_kfold[fold_num][:,0],train_pos_kfold[fold_num][:,1]]
train_neg_score = score_mat[train_neg_kfold[fold_num][:,0],train_neg_kfold[fold_num][:,1]]
test_pos_score = score_mat[test_pos_kfold[fold_num][:,0],test_pos_kfold[fold_num][:,1]]
test_neg_score = score_mat[test_neg_kfold[fold_num][:,0],test_neg_kfold[fold_num][:,1]]

# Running time

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris(as_frame=True)
df = iris.data.assign(target=iris.target_names[iris.target])
df

In [ ]:
from weave.monitoring import StreamTable

In [ ]:
api = wandb.Api()
done_methods = ['KG4SL','NSF4SL','PTGNN','SLMGAE','PiLSL','CMFW','DDGCN','SL2MF','GRSMF','GCATSL','MGE4SL']
# done_methods = ['PiLSL']
runs = api.runs('slbench/Benchmarking')

In [ ]:
runs[0].scan_history(keys = ['test_M10'])

In [ ]:
StreamTable(runs[0].history)